# Dataset

In [1]:
import sys

# add the parent directory to the path so we can import the module
sys.path.append("/data2/eranario/scratch/strawberry-yield-forecasting")

In [2]:
import torch
from src.dataset import StrawberryDataset

In [3]:
# create random seed
torch.manual_seed(126)

In [4]:
# device is cuda else cpu
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
path_to_counts = "/data2/eranario/data/Strawberry-Yield-Forecasting/"
path_to_weights = "/data2/eranario/data/Strawberry-Yield-Forecasting/weights/weights.csv"
n_seq = 5
seq_l = 3
n_folds = 2
k_fold = 2
use_weights = True
dataset = StrawberryDataset(path_to_counts, path_to_weights, k_fold=k_fold,
                            n_seq=n_seq, seq_l=seq_l, n_folds=n_folds, use_weights=use_weights)

In [6]:
# Check date parsing
months, days, years = dataset.months, dataset.days, dataset.years
print("\nParsed Dates:")
print("Months:", months)
print("Days:", days)
print("Years:", years)


X_data, y_data = dataset.fnX, dataset.fny
print("\nOrganized Data Shapes:")
print("X_data shape:", X_data.shape)
print("y_data shape:", y_data.shape)

# Display sample data from X and y
print("\nSample X_data:", X_data[0][:10])  # Display first 10 features of first sample
print("Sample y_data:", y_data[0])         # Display first sample of y_data

print("\nDataset length (number of samples):", len(dataset))

X_sample, y_sample = dataset[0]
print("\nSample from __getitem__:")
print("X_sample:", X_sample[:10])  # Display first 10 features of X_sample
print("y_sample:", y_sample)


Parsed Dates:
Months: ['06', '06', '07', '07', '07', '07', '07', '07', '08', '08', '08', '08', '08', '09', '09', '09']
Days: ['17', '28', '05', '08', '15', '19', '26', '29', '02', '05', '09', '12', '29', '01', '20', '22']
Years: ['2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022']

Organized Data Shapes:
X_data shape: (495, 3, 7)
y_data shape: (495, 1)

Sample X_data: [[0.         0.25957873 0.08863333 0.         0.         0.56364411
  1.        ]
 [0.         0.         0.         0.         0.         0.
  0.47619048]
 [0.         0.         0.         0.3789208  0.         0.34544651
  0.14285714]]
Sample y_data: [0.33320494]

Dataset length (number of samples): 495

Sample from __getitem__:
X_sample: tensor([[0.0000, 0.2596, 0.0886, 0.0000, 0.0000, 0.5636, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4762],
        [0.0000, 0.0000, 0.0000, 0.3789, 0.0000, 0.3454, 0.1429]])
y_sample:

# Dataloader

In [7]:
import torch
from torch.utils.data import DataLoader

In [8]:
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [9]:
len(train_loader), len(val_loader)

(93, 31)

In [10]:
# try to get a batch of data
for i, (X_batch, y_batch) in enumerate(train_loader):
    print("\nBatch", i)
    print("X_batch shape:", X_batch.shape)
    print("y_batch shape:", y_batch.shape)
    print("X_batch:", X_batch) # (batch_size, n_seq, num_classes)
    print("y_batch:", y_batch) # (batch_size, 1)
    break


Batch 0
X_batch shape: torch.Size([4, 3, 7])
y_batch shape: torch.Size([4, 1])
X_batch: tensor([[[0.0000, 0.0709, 0.4952, 0.0000, 0.0000, 0.9426, 1.0000],
         [0.0000, 0.0000, 0.5245, 0.1726, 0.0000, 0.6923, 0.7857],
         [0.0000, 0.0000, 0.0616, 0.5772, 0.0000, 0.8210, 0.2857]],

        [[0.0000, 0.0648, 0.7097, 0.0000, 0.0000, 0.8716, 1.0000],
         [0.0000, 0.0000, 0.5942, 0.1149, 0.0000, 0.7507, 0.7857],
         [0.0000, 0.0000, 0.0703, 0.6614, 0.0000, 0.8628, 0.2857]],

        [[0.0000, 0.4654, 0.0981, 0.0000, 0.0000, 0.8405, 1.0000],
         [0.0000, 0.0000, 0.2981, 0.1757, 0.0000, 0.7615, 0.4762],
         [0.0000, 0.0000, 0.0000, 0.2106, 0.0000, 0.2322, 0.1429]],

        [[0.0000, 0.3142, 0.0475, 0.0000, 0.0000, 0.3085, 1.0000],
         [0.0000, 0.0000, 0.1295, 0.0469, 0.0000, 0.1957, 0.4762],
         [0.0000, 0.0000, 0.0000, 0.0772, 0.0000, 0.1404, 0.1429]]])
y_batch: tensor([[0.8986],
        [0.7995],
        [0.3200],
        [0.1485]])


# Training

In [11]:
import torch
import matplotlib.pyplot as plt

from torch.nn import MSELoss
from torch.optim import Adam
from src.model import Transformer

In [12]:
def train_transformer_decoder(model, train_loader, val_loader, epochs, lr):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = MSELoss()
    optimizer = Adam(model.parameters(), lr=lr)

    best_val_loss = float("inf")
    best_model = None

    train_losses = []  # To store train loss for each epoch
    val_losses = []    # To store validation loss for each epoch

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Generate causal mask
            seq_len = X_batch.size(1)
            causal_mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1).bool()

            optimizer.zero_grad()
            outputs = model(X_batch, causal_mask=causal_mask)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)  # Append to train losses

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                # Generate causal mask
                seq_len = X_batch.size(1)
                causal_mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1).bool()

                outputs = model(X_batch, causal_mask=causal_mask)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)  # Append to validation losses

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()

    # Load best model before returning
    model.load_state_dict(best_model)

    # Plot training and validation loss curves
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, epochs + 1), train_losses, label="Train Loss")
    plt.plot(range(1, epochs + 1), val_losses, label="Validation Loss", linestyle="--")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Train vs Validation Loss Curve")
    plt.legend()
    plt.tight_layout()
    plt.show()

    return model

In [13]:
input_dim = dataset.samples_dim[2]
epochs = 50
learning_rate = 0.0001

model = Transformer(input_dim=input_dim, hidden_size=12, num_heads=4, num_layers=3)

In [ ]:
trained_model = train_transformer_decoder(model, train_loader, val_loader, epochs, learning_rate)

Epoch 1/50, Train Loss: 0.2267, Val Loss: 0.0273
Epoch 2/50, Train Loss: 0.0433, Val Loss: 0.0175
Epoch 3/50, Train Loss: 0.0337, Val Loss: 0.0145
Epoch 4/50, Train Loss: 0.0240, Val Loss: 0.0148
Epoch 5/50, Train Loss: 0.0211, Val Loss: 0.0161
Epoch 6/50, Train Loss: 0.0210, Val Loss: 0.0160
Epoch 7/50, Train Loss: 0.0190, Val Loss: 0.0159
Epoch 8/50, Train Loss: 0.0200, Val Loss: 0.0158
Epoch 9/50, Train Loss: 0.0178, Val Loss: 0.0152


# Test

In [ ]:
import numpy as np

In [ ]:
def evaluate_test_set(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    predictions = []
    true_values = []
    
    with torch.no_grad():  # Disable gradient computation for inference
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            predictions.append(outputs.cpu().numpy())  # Store predictions
            true_values.append(y_batch.cpu().numpy())  # Store true labels
    
    # Concatenate the results into single arrays
    predictions = np.concatenate(predictions)
    true_values = np.concatenate(true_values)
    
    return predictions, true_values


In [ ]:
dataset.mode = 'test'

test_loader = DataLoader(dataset, batch_size=1, shuffle=False)

# check first batch
X_batch, y_batch = next(iter(test_loader))
print("\nSample from test_loader:")
print("X_batch shape:", X_batch.shape)
print("y_batch shape:", y_batch.shape)
print("X_batch:", X_batch)

In [ ]:
predictions, true_values = evaluate_test_set(trained_model, test_loader, device)

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress

# change true and predicted from (495, 1) to (5. 99)
true_values_reshaped = true_values.reshape(n_seq, -1)
predictions_reshaped = predictions.reshape(n_seq, -1)

scatter_marker = ['.','d','*','x','+','o','.','x','d','*']
scatter_color = ['black','gray','orange','steelblue','darkviolet','blue','pink','blue','pink','coral']

time_folds = [[4, 5, 6, 7, 8], [11, 12, 13, 14, 15]]

plt.figure()
r2_list = []
rmse_list = []

for time in range(n_seq):
    slope, intercept, r_value, p_value, std_err = linregress(true_values_reshaped[time, :], predictions_reshaped[time, :])
    r2 = r_value ** 2
    rmse = np.sqrt(mean_squared_error(true_values_reshaped[time], predictions_reshaped[time]))
    
    label = str(time_folds[k_fold-1][time])
    plt.scatter(true_values_reshaped[time, :], predictions_reshaped[time, :], marker = scatter_marker[time], color=scatter_color[time], label = f'$t_{{{label}}}$, $R^2$= {r2:.2f}, RMSE= {rmse:.2f}')
    r2_list.append(r2)
    rmse_list.append(rmse)

print(f"R^2: {np.mean(r2_list):.4f}", f"RMSE: {np.mean(rmse_list):.4f}")
plt.xlabel('True yield (Norm.)')
plt.ylabel('Predicted yield (Norm.)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Access the scaler
y_scaler = dataset.y_scaler

# Determine the number of expected features
expected_features = y_scaler.min_.shape[0]

# Pad true values and predictions
true_values_padded = np.zeros((true_values.shape[0], expected_features))
predictions_padded = np.zeros((predictions.shape[0], expected_features))

# Assign the actual values to the appropriate column (e.g., first column)
true_values_padded[:, 0] = true_values.flatten()
predictions_padded[:, 0] = predictions.flatten()

# Unnormalize and extract the relevant column
true_values_unnormalized = y_scaler.inverse_transform(true_values_padded)[:, 0]
predictions_unnormalized = y_scaler.inverse_transform(predictions_padded)[:, 0]

# Reshape for evaluation
true_values_reshaped_final = true_values_unnormalized.reshape(n_seq, -1)
predictions_reshaped_final = predictions_unnormalized.reshape(n_seq, -1)

scatter_marker = ['.','d','*','x','+','o','.','x','d','*']
scatter_color = ['black','gray','orange','steelblue','darkviolet','blue','pink','blue','pink','coral']

time_folds = [[4, 5, 6, 7, 8], [11, 12, 13, 14, 15]]

plt.figure()
r2_list = []
rmse_list = []

for time in range(n_seq):
    slope, intercept, r_value, p_value, std_err = linregress(
        true_values_reshaped_final[time, :], predictions_reshaped_final[time, :]
    )
    r2 = r_value ** 2
    rmse = np.sqrt(mean_squared_error(
        true_values_reshaped_final[time], predictions_reshaped_final[time]
    ))

    label = str(time_folds[k_fold-1][time])
    plt.scatter(
        true_values_reshaped_final[time, :], predictions_reshaped_final[time, :],
        marker=scatter_marker[time], color=scatter_color[time],
        label=f'$t_{{{label}}}$, $R^2$= {r2:.2f}, RMSE= {rmse:.2f}'
    )
    r2_list.append(r2)
    rmse_list.append(rmse)

print(f"R^2: {np.mean(r2_list):.4f}", f"RMSE: {np.mean(rmse_list):.4f}")
plt.xlabel('True yield')
plt.ylabel('Predicted yield')
plt.legend()
plt.tight_layout()
plt.show()
